# PROJECT: Modeling Magnetic Resonance using QuTiP
### **Maastricht Science Programme, Project Period January 2022**
### **Supplementary Notebook**

## Table of Contents

## Introduction

This notebook will guide the user through several quantum mechanics concepts using various different interactive animations. These concepts include **visualizing H-orbitals** and **magnetic fields**.

##### References
- https://dpotoyan.github.io/Chem324/H-atom-wavef.html
- https://demonstrations.wolfram.com/HydrogenAtomRadialFunctions/
- https://github.com/kimfetti/Videos/blob/master/ipywidgets/03_quick_animation.ipynb

## Libraries

To start this notebook, libraries and their features need to be imported.

In [13]:
from qutip import *
import numpy as np
from pylab import *
import matplotlib.pyplot as plt
from matplotlib import cm, colors
from mpl_toolkits.mplot3d import Axes3D
import ipywidgets as widgets
import scipy.special as spe
from mpl_toolkits import mplot3d

## Radial plots and orbitals

Using Laguerre special functions from the SciPy library a radial plot and radial probability plot are generated. The user can change the input for the principle (n) and the orbital (l) quantum numbers by adjusting the sliders.

In [2]:
#Radial part of the wavefunction (R10 --> a0=a^(3/2))
#Uses the special functions where Laguerre polynomials are defined 
def psi_R(r,n,l):

    w = np.sqrt((2.0/n)**3 * spe.factorial(n-l-1) /(2.0*n*spe.factorial(n+l)))
    
    x = spe.assoc_laguerre(2.0*r/n,n-l-1,2*l+1)
    
    y = np.exp(-r/n)
    
    z = (2.0*r/n)**l
    
    answer = w*x*y*z
    return answer

In [17]:
def radial_function (n, l=0):
    x = np.linspace(0,10,100) 
    y = psi_R(x,n,l) 
    E_y = np.absolute((y**2))*(x**2)
    
    #Radial function 
    plt.subplot(211)
    plt.plot(x, y**2, lw=3)
    plt.xlabel('$r [a_0]$',fontsize=20)
    plt.ylabel('$R_{nl}(r)$', fontsize=20)
    plt.grid('True')
    
    #Radial probability density plot 
    plt.subplot(212)
    plt.plot(x, E_y, color='orange', lw=3, )
    plt.xlabel('$r [a_0]$',fontsize=20)
    plt.ylabel('$r^2R^2_{nl}(r)$', fontsize=20)
    plt.grid('True')
    
widgets.interact(radial_function, n=(1,5,1), l=(0,5,1))


interactive(children=(IntSlider(value=3, description='n', max=5, min=1), IntSlider(value=0, description='l', m…

<function __main__.radial_function(n, l=0)>

Using the spherical harmonics function from the SciPy library the angular part of the hydrogen wavefunction can be generated. The user can adjust the input for the principle (n) and magnetic quantum number with the widgets below.

In [7]:
def psi_ang(m, n, theta, phi):
    
    sphHarm = spe.sph_harm(m, n, theta, phi) 
    #theta in [0, 2pi] and phi in [0, pi]
    #computes spherical harmonics using associated Legendre functions (Pnm)
    print(sphHarm.real)
    return sphHarm.real

In [8]:
# computes the spherical harmonics on a grid
phi, theta = np.linspace(0, np.pi, 100), np.linspace(0, 2*np.pi, 100)
phi, theta = np.meshgrid(phi, theta) #creates rectangular grid out of phi and theta values

In [9]:
print("Lets compute spherical harmonics of the H atom.")

def Quantum_number (yn, ym):
    
    Ymn = psi_ang(yn, ym, theta, phi)
    #phi and theta array

    # spherical to Cartesian coordinates 
    x = np.sin(phi) * np.cos(theta) * abs(Ymn)
    y = np.sin(phi) * np.sin(theta) * abs(Ymn)
    z = np.cos(phi) * abs(Ymn)
    
    fig = plt.figure(figsize=(10,10))
    ax = fig.add_subplot(111, projection='3d')

    ''' Normalize color bar to [0,1] scale'''

    fcolors = (Ymn - Ymn.min())/(Ymn.max() - Ymn.min())

    '''Make 3D plot of real part of spherical harmonic'''

    ax.plot_surface(x, y, z, facecolors=cm.seismic(fcolors), alpha=0.3)


    ''' Set axes limit to keep aspect ratio 1:1:1 '''

    ax.set_xlim(-1, 1)
    ax.set_ylim(-1, 1)
    ax.set_zlim(-1, 1)
    
    

widgets.interact(Quantum_number, yn=(0,4,1), ym=(0,4,1))

Lets compute spherical harmonics of the H atom.


interactive(children=(IntSlider(value=2, description='yn', max=4), IntSlider(value=2, description='ym', max=4)…

<function __main__.Quantum_number(yn, ym)>

## Static and oscillating magnetic field

This part of the notebook will introduce an interactive visualization of spin 1/2 particle placed into a magnetic field. The magnetic field is made up of a static and an oscillating magnetic field. The static component is in the z-direction and the oscillating component is along the x-direction. 

The static magnetic field along the z-direction is plotted below. The strength of the field is denoted by Bz and can be adjusted using the sliders. The maximum of the magnetic field can be determined by the user.

In [11]:
#Lets place our spin-1/2 particle in a magnetic field.
print("What should the maximum field strength along the z-axis be?")

A = float(input("Bz = "))
def magneticfield_z(Bz): #Function to plot our static magnetic field
    lines_plotted = plt.plot([]) #Empty plot
    plt.ylim(0,(A + 1)) #Want y-limit to always be slightly higher than the input max, so that we can see the field line
    line_plotted = lines_plotted[0]
    plt.xlim(0,2*np.pi)
    x = np.linspace(0,2*np.pi,100)
    line_plotted.set_data((x, Bz))
    
widgets.interact(magneticfield_z, Bz=(0.0, A, 1.0)) 



What should the maximum field strength along the z-axis be?
Bz = 5


interactive(children=(FloatSlider(value=2.0, description='Bz', max=5.0, step=1.0), Output()), _dom_classes=('w…

<function __main__.magneticfield_z(Bz)>

The oscillating component along the x-axis is plotted below. The magnetic field strength is denoted with Bx and can be adjusted by the user with the widget. The magnetic field oscillates at the angular frequency (w), which also can be adjusted by the user. The maximum of both the magentic field strength and the angular frequency can be determined by the user.

This will generate an animation that loops over time, which can be played by clicking the play button. Clicking the loop button will create an animation that loops indefinitely. By playing the animation and adjusting the field strength and the angular frequency, the user can see how this changes the magnetic field.

In [12]:
#Lets place our spin-1/2 particle in a magnetic field.

print("What should the maximum field strength along the x-axis be?")
C = float(input("Bx = "))

print("What should the maximum of the angular frequency be?")
D = float(input("The angular frequency is: "))

def plot(Bx=4.0, w=4.0, t=0.0):
    lines_plotted = plt.plot([])

    plt.ylim(-10, 10)
    line_plotted = lines_plotted[0]
    plt.xlim(0, 2 * np.pi)

    x = np.linspace(0, 2 * np.pi, 100)
    B_x = 0

    B_x = C * np.cos(x + w * t)  
    line_plotted.set_data((x, B_x))



# widget that slides for amplitude and frequency and loops for time

# w = increments of half pi
# click on loop button to loop over time otherwise it lasts 30 sec
widgets.interact(plot, Bx=(0.0, C, 1.0), w=(0.0, D, np.pi / 2), t=widgets.Play(min=0.0, max=30.0))

What should the maximum field strength along the x-axis be?
Bx = 6
What should the maximum of the angular frequency be?
The angular frequency is: 8


interactive(children=(FloatSlider(value=4.0, description='Bx', max=6.0, step=1.0), FloatSlider(value=4.0, desc…

<function __main__.plot(Bx=4.0, w=4.0, t=0.0)>

The static and the oscillating magnetic field act on the spin 1/2 particle at the same time. This is visualized in the plot below. The spin 1/2 plot is denoted by a sphere in the middle

The user can adjust the slides to change the strenght of the magnetic field in both the z-direction (Bz) and the x-direction (Bx), the user can also change the angular frequency (w) for the oscillating magnetic field. Again an animation is made that loops over time, which can be played by clicking the play button, or looped over indefinitely by clicking the loop button. While the animation plays the magnetic fields and the angular frequency can be adjusted.

In [16]:
def plot_3D (Bx, w, Bz, t):
    
    fig = plt.figure()
    ax = fig.add_subplot(111, projection ='3d')

    # defining 3 axes for oscillating field
    x1 = np.linspace(-10,10,50)
    y1 = np.linspace(-10,10,50)
    z1 = Bx* np.cos(y1+w*t) 

    # np.zeros_like makes sure the x-axis is zero, to get a 2D plot
    ax.plot(np.zeros_like(z1),y1,z1)

    #defining axis for static field
    z2=np.linspace(-(Bz),Bz,50)

    ax.plot(np.zeros_like(z2),np.zeros_like(z2),z2)

    # adding dot that could represent a hydrogen atom
    ax.scatter(0,0,0, s=500, c='pink')

    # limits on axes
    ax.set_xlim(-10,10)
    ax.set_ylim(-10,10)
    ax.set_zlim(-10,10)

    # axis labels
    ax.set_xlabel("x")
    ax.set_ylabel("y")
    ax.set_zlabel("z")


    # plotting
    fig.set_size_inches(7, 7)
    plt.show()
    
    
widgets.interact(plot_3D, Bx=(0.0, 10, 1.0), w=(0.0, 10, np.pi / 2), Bz=(0.0, 10, 1.0), t=widgets.Play(min=0.0, max=30.0))

interactive(children=(FloatSlider(value=5.0, description='Bx', max=10.0, step=1.0), FloatSlider(value=4.712388…

<function __main__.plot_3D(Bx, w, Bz, t)>